# Twitter Analysis 

* getting tweets

* cleaning Tweets

* saving in MongoDB

* Sentiment Analysis using TextBlob

### Importing libraries

In [343]:

import tweepy
from pymongo import MongoClient
from pprint import pprint
import re
from textblob import TextBlob
import pandas as pd
import matplotlib.pyplot as plt
import jdc 



### Initial Class

Class Tweet retrieve Tweets

*  load_api() to initialise Twitter App
*  create_db() to create database and retrieve tweets  from the mongoDB 
*  get_tweet() to query 
*  save_tweet() to save tweets

In [ ]:
class Tweet:

    #Api_call
    def load_api(self):
        CONSUMER_KEY = 'Consumer Key'
        CONSUMER_SECRET = 'Consumer secret'
        oAuthToken = 'Authorisation Token'
        OAUTH_TOKEN_SECRET= 'authorisation token secret key'
        auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
        auth.set_access_token(oAuthToken, OAUTH_TOKEN_SECRET)
        return tweepy.API(auth) 
    
    #creating a db in mongodb
    def create_db(self):
        self.client = MongoClient()
        self.dbNames=self.client.list_database_names()
        if "tweet_db" in self.dbNames:
            self.db=self.client.tweet_db
            self.tcollection=self.db.tweet_collection
            self.pointer=self.tcollection.find({})
            return self.pointer
            
        else:
            self.db=self.client.tweet_db
            self.t_collection=self.db.tweet_collection
            self.t_collection.create_index([("id",pymongo.ASCENDING)],unique=True)
            self.get_tweet()
        
        #query needed for the api call
    def get_tweet(self):
        self.query="ozil"
        self.Ap=self.load_api()
        self.results=self.Ap.search(q= self.query)
        #for r in results:
            #print(r)
        self.save_tweet()
    
    #inserting required attributes in db
    def save_tweet(self):
        for r in self.results:
            self.individual_tweet=({"id":r.id,"user_id":r.user.id,"tweet_text":r.text,"created_at":r.created_at})
            self.t_collection.insert_one(self.individual_tweet)
        #pointer = self.t_collection.find({})
        #for p in pointer: 
         #   pprint(p)
   
    

### Class to Clean the data
*  clean() cleaning the data by removing hashtags,@, Urls, emojis etc

In [342]:

class Data_Cleaner(Tweet):
    
    def clean(self):
        self.db_pointer=Tweet.create_db(self)     
        try:
 
            #cleaning the data
            for p in self.db_pointer:
                temp_tweet=p["tweet_text"]
                        #removing hashtags replacement #
                p["tweet_text"] = re.sub(r'#\w+ ?', '#', p['tweet_text'])
                     #removing urls replacement URL
                p["tweet_text"] = re.sub(r'http\S+', 'URL', p['tweet_text'])
                        #removing tags -replacement @
                p["tweet_text"] = re.sub(r'@[a-zA-Z0-9]+','@', p['tweet_text'])
                    
                p["tweet_text"] = re.sub(r'@_[a-zA-Z0-9]_+','@', p['tweet_text'])
                        
                    #removing emojis-replacement **
                                          
                remove_emoji = re.compile(u'['
                                                    u'\U0001F300-\U0001F64F'
                                                    u'\U0001F680-\U0001F6FF'
                                                    u'\u2600-\u26FF\u2700-\u27BF]+', 
                                                    re.UNICODE)
                p["tweet_text"]=remove_emoji.sub("**",p["tweet_text"])
                    
                
                #print(p["tweet_text"])
                myquery= {"tweet_text":temp_tweet}
                newvalues={"$set":{"tweet_text":p["tweet_text"]}}
                tweet_len={"$set":{"len":len(p["tweet_text"])}}
                self.tcollection.update_one(myquery,newvalues)
                self.tcollection.update_one(myquery,tweet_len)
                     
        except:
            print("Error while Cleaning the data")
            pass
        

            


### Class to analyse the Tweets
*  analyze() uses the TextBlob to find the sentiment and catagorise the tweets into positiv,negative or neutral and also updating the Mongo collection

In [ ]:
class Data_Analyser(Data_Cleaner):
    
    def analyze(self):
        
        polarity_list = []
        tweet1 = []
        number = 1
        db_pointer=Tweet.create_db(self)
        df=pd.DataFrame(list(db_pointer))
        #print(df["tweet_text"])
        for item in df["tweet_text"] :
            #tweet=Tweet.clean(self,item)
            try:
                analysis = TextBlob(item)
                analysis = analysis.sentiment
                polarity = analysis.polarity
                tweet1.append(item)
                    
                if(polarity>0):
                    sentiment_value= 'positive'
                elif(polarity==0):
                    sentiment_value ='neutral'
                else:
                    sentiment_value= 'negative'    
                    
                polarity_list.append(sentiment_value) 
                
                myquery= {"tweet_text":item}
                newvalues={"$set":{"Sentiment Value":sentiment_value}}
                self.tcollection.update_one(myquery,newvalues)
            except:
                print("errorr")
        
        
        #outputDF=pd.DataFrame({'Tweet':tweet_list,'Sentiment_value':polarity_list})
        #print(outputDF)
        


    

                                     
    

### main()

In [ ]:

if __name__ == '__main__':
    t=Tweet()
    t.create_db()
    dc=Data_Cleaner()
    dc.clean()
    da=Data_Analyser()
    da.analyze()